Author:
Zvi Lapp

# Unsupervised Neural Network Learning

In this notebook will use denoising autoencoder on mnist data sent to learn a feature extractor and then use knn algorithm to predict an images class and will use tsne algo to visualize the predictions as well

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import math
import pylab as Plot
import os
import shutil
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
import matplotlib.pyplot as plt
from libs.utils import corrupt
import platform
import math
import operator 
from sklearn.manifold import TSNE
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

In [ ]:
#should be 3.5.2
platform.python_version()

## Autoencoder

The autoencoder is of dimensions 784, 512, 256, 30, 256, 512, 784 and the goal is to have the neural network reproduce (day dream) the input image in order to have use the layer of 30 as feature extractor. Meaning the training done to reproduce the digit will be used as a image to vec high level feature extractor.

In [ ]:
def autoencoder(dimensions=[784, 512, 256, 30]):

    # input to the network
    x = tf.placeholder(tf.float32, [None, dimensions[0]], name='x')

    corrupt_prob = tf.placeholder(tf.float32, [1])
    current_input = corrupt(x) * corrupt_prob + x * (1 - corrupt_prob)

    # Build the encoder
    encoder = []
    for layer_i, n_output in enumerate(dimensions[1:]):
        n_input = int(current_input.get_shape()[1])
        W = tf.Variable(
            tf.random_uniform([n_input, n_output],
                              -1.0 / math.sqrt(n_input),
                              1.0 / math.sqrt(n_input)))
        b = tf.Variable(tf.zeros([n_output]))
        encoder.append(W)
        output = tf.nn.tanh(tf.matmul(current_input, W) + b)
        current_input = output
    # latent representation
    z = current_input
    encoder.reverse()
    # Build the decoder using the same weights
    for layer_i, n_output in enumerate(dimensions[:-1][::-1]):
        W = tf.transpose(encoder[layer_i])
        b = tf.Variable(tf.zeros([n_output]))
        output = tf.nn.tanh(tf.matmul(current_input, W) + b)
        current_input = output
    # now have the reconstruction through the network
    y = current_input
    # cost function measures pixel-wise difference
    cost = tf.sqrt(tf.reduce_mean(tf.square(y - x)))
    return {'x': x, 'z': z, 'y': y,
            'corrupt_prob': corrupt_prob,
            'cost': cost}

In [ ]:
def get_label(y):
    for i in range(len(y)):
        if y[i]:
            return i
    return 0

def train_mnist(start_epoch=0, n_epochs = 10000, restore=False, save=True):   
    
    # load MNIST as before
   
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
    mean_img = np.mean(mnist.train.images, axis=0)
    ae = autoencoder(dimensions=[784, 512, 256, 30])
    

    
    learning_rate = 0.001
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(ae['cost'])

    
    # We create a session to use the graph
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    
    
    directory='saved_models'
    if restore:
        print("Restoring")
        saver.restore(sess,'./'+directory+'/model.ckpt-'+str(start_epoch))
        print("Restored")

    
    # Fit all training data
    batch_size = 50
    
    for epoch_i in range(start_epoch,n_epochs):
        for batch_i in range(mnist.train.num_examples // batch_size):
            batch_xs, _ = mnist.train.next_batch(batch_size)
            train = np.array([img - mean_img for img in batch_xs])
            sess.run(optimizer, feed_dict={
                ae['x']: train, ae['corrupt_prob']: [1.0]})
        print(epoch_i, sess.run(ae['cost'], feed_dict={
            ae['x']: train, ae['corrupt_prob']: [1.0]}))
        #verify(sess)
        if save:
            if ((epoch_i%10)==0):
                saver.save(sess, directory+'/model.ckpt',global_step=epoch_i)
    
    # Plot example reconstructions
    n_examples = 15
    test_xs, _ = mnist.test.next_batch(n_examples)
    test_xs_norm = np.array([img - mean_img for img in test_xs])
    recon = sess.run(ae['y'], feed_dict={
        ae['x']: test_xs_norm, ae['corrupt_prob']: [0.0]})
    fig, axs = plt.subplots(2, n_examples, figsize=(10, 2))
    for example_i in range(n_examples):
        axs[0][example_i].imshow(
            np.reshape(test_xs[example_i, :], (28, 28)))
        axs[1][example_i].imshow(
            np.reshape([recon[example_i, :] + mean_img], (28, 28)))
    fig.show()
    plt.draw()
    
    #predict on test
    test_xs, test_ys = mnist.test.next_batch(mnist.test.num_examples)


    hidden = sess.run(ae['z'], feed_dict={
             ae['x']: test_xs, ae['corrupt_prob']: [0.0]})
    
    #prep for knn
    nn=[]
    for h,y in zip(hidden,test_ys):
        l = h.tolist()+[get_label(y.tolist())]
        nn.append(l)
        
    return nn,hidden,test_ys


## KNN

Algorithm to get euclidean distance to nearest neighbor and predict label.

In [ ]:
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes

def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        #print("Label",testSet[x][-1])
        #print("Prediction",predictions[x][0][0])
        if testSet[x][-1] is predictions[x][0][0]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

def knn(nn):
    print(len(nn))
    predictions=[]
    k=2
    for x in range(len(nn)):   
        neighbors = getNeighbors(nn, nn[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        print('>',x,'predicted=' + repr(result) + ', actual=' + repr(nn[x][-1]))
    accuracy = getAccuracy(nn, predictions)
    print('Accuracy: ' + repr(accuracy) + '%')

## TSNE

Algorithm to visualize graphical representaiton of 30 dim vector embeddings in lower dim.

In [ ]:
def plot_with_labels(lowDWeights, labels, filename='tsne.png'):
    assert lowDWeights.shape[0] >= len(labels), "More labels than weights"
    plt.figure(figsize=(20, 20))  #in inches
    for i, label in enumerate(labels):
        x, y = lowDWeights[i,:]
        plt.scatter(x, y)
        plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

    plt.savefig(filename)
        
def tsne(X,labels):
    print("Computing t-SNE")
    tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
    plot_only = 2000
    X_tsne = tsne.fit_transform(X[0:plot_only,:])
    l = [str(get_label(l_i)) for l_i in labels[0:plot_only]]
    
    plot_with_labels(X_tsne, l, "t-SNE embedding of the digits")

    #plt.show()

## Main

Beginning process

In [ ]:
if __name__ == '__main__':
    nn,X,labels = train_mnist(start_epoch=0,n_epochs = 10, restore=False, save=True)
    knn(nn)
    tsne(X,labels)